In [1]:
import pandas as pd
from pathlib import Path

In [3]:
path = "COVID-19/csse_covid_19_data/csse_covid_19_daily_reports"
all_files = Path(path).glob("*.csv") 

df = pd.concat(map(pd.read_csv, all_files))

In [5]:
df.shape

(2645022, 21)

In [6]:
df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,...,Combined_Key,Incidence_Rate,Case-Fatality_Ratio,Incident_Rate,Case_Fatality_Ratio,Province/State,Country/Region,Last Update,Latitude,Longitude
0,45001.0,Abbeville,South Carolina,US,2020-04-29 02:32:29,34.223334,-82.461707,29.0,0.0,0.0,...,"Abbeville, South Carolina, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22001.0,Acadia,Louisiana,US,2020-04-29 02:32:29,30.295065,-92.414197,130.0,9.0,0.0,...,"Acadia, Louisiana, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,51001.0,Accomack,Virginia,US,2020-04-29 02:32:29,37.767072,-75.632346,223.0,4.0,0.0,...,"Accomack, Virginia, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16001.0,Ada,Idaho,US,2020-04-29 02:32:29,43.452658,-116.241552,662.0,16.0,0.0,...,"Ada, Idaho, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19001.0,Adair,Iowa,US,2020-04-29 02:32:29,41.330756,-94.471059,1.0,0.0,0.0,...,"Adair, Iowa, US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def remove_small_countries(df):
    countries = df.groupby("Country_Region").last().query('Confirmed > 100000').index.to_list()
    return df[df.Country_Region.isin(countries)]

In [8]:
def report_shape(df):
    print(df.shape)
    return df

In [ ]:
(df
 .assign(Last_Update = pd.to_datetime(df["Last_Update"]).dt.date)
 .loc[:, ["Country_Region", "Last_Update", "Confirmed"]]
 .dropna()
 .groupby(["Country_Region", "Last_Update"])
 .sum()
 .reset_index()
 .set_index("Last_Update")
 .sort_index()
 .pipe(remove_small_countries)
 .pipe(report_shape)
 .to_parquet("tmp/data.parquet")
)

NameError: name 'df' is not defined